In [22]:
# MLP for Pima Indians Dataset Serialize to JSON and HDF5
import numpy as np
import os
import cv2
from collections import deque
import pafy
import math
import random
import datetime as dt
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from moviepy.editor import *
%matplotlib inline
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [29]:
SEQUENCE_LENGTH = 40
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
CLASSES_LIST = ["normal", "nude", "Violence"]

In [24]:
# load json and create model
json_file = open('LRCN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
LRCN_model = model_from_json(loaded_model_json)
# load weights into new model
LRCN_model.load_weights("LRCN_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [90]:
!pip3 install pytube==6.4.2

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [108]:
# importing the module 
from pytube import YouTube 
  
# where to save 
SAVE_PATH = "test_videos_1" #to_do 
  
# link of the video to be downloaded 
link="https://www.youtube.com/watch?v=ZAt8t8NWcWU"
yt = YouTube(link)
stream = yt.streams.first()
out_file = stream.download(SAVE_PATH)
print(out_file)
os.rename(out_file,SAVE_PATH+'/'+'test_video.mp4')

/home/subham/Desktop/upwork/project30/obscene-video-recognition/approach2/test_videos_1/Playing with balls ideas for children.3gpp


In [109]:
# Make the Output directory if it does not exist
test_videos_directory = 'test_videos_1'
os.makedirs(test_videos_directory, exist_ok = True)
video_title='test_video'
# Download a YouTube Video.
#yt_download("",test_videos_directory+'/'+video_title+'.mp4')

# Get the YouTube Video's path we just downloaded.
input_video_file_path = f'{test_videos_directory}/{video_title}.mp4'

In [110]:
def predict_on_video(video_file_path, output_file_path, SEQUENCE_LENGTH):
    '''# Construct the output video path.
output_video_file_path = f'{test_videos_directory}/{video_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4'
 
# Perform Action Recognition on the Test Video.
predict_on_video(input_video_file_path, output_video_file_path, SEQUENCE_LENGTH)
 
# Display the output video.
VideoFileClip(output_video_file_path, audio=False, target_resolution=(300,None)).ipython_display()
    This function will perform action recognition on a video using the LRCN model.
    Args:
    video_file_path:  The path of the video stored in the disk on which the action recognition is to be performed.
    output_file_path: The path where the ouput video with the predicted action being performed overlayed will be stored.
    SEQUENCE_LENGTH:  The fixed number of frames of a video that can be passed to the model as one sequence.
    '''
 
    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)
 
    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # Initialize the VideoWriter Object to store the output video in the disk.
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 
                                   video_reader.get(cv2.CAP_PROP_FPS), (original_video_width, original_video_height))
    print(output_file_path)
    # Declare a queue to store video frames.
    frames_queue = deque(maxlen = SEQUENCE_LENGTH)
 
    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''
 
    # Iterate until the video is accessed successfully.
    while video_reader.isOpened():
 
        # Read the frame.
        ok, frame = video_reader.read() 
        
        # Check if frame is not read properly then break the loop.
        if not ok:
            break
 
        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255
 
        # Appending the pre-processed frame into the frames list.
        frames_queue.append(normalized_frame)
 
        # Check if the number of frames in the queue are equal to the fixed sequence length.
        if len(frames_queue) == SEQUENCE_LENGTH:
 
            # Pass the normalized frames to the model and get the predicted probabilities.
            predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_queue, axis = 0))[0]
 
            # Get the index of class with highest probability.
            predicted_label = np.argmax(predicted_labels_probabilities)
            
            # Get the class name using the retrieved index.
            predicted_class_name = CLASSES_LIST[predicted_label]
            print(predicted_labels_probabilities, predicted_class_name) 
        # Write predicted class name on top of the frame.
        cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
 
        # Write The frame into the disk using the VideoWriter Object.
        video_writer.write(frame)
        
    # Release the VideoCapture and VideoWriter objects.
    video_reader.release()
    video_writer.release()

In [111]:
# Construct the output video path.
output_video_file_path = f'{test_videos_directory}/{video_title}-Output-SeqLen{SEQUENCE_LENGTH}.mp4'
 
# Perform Action Recognition on the Test Video.
predict_on_video(input_video_file_path, output_video_file_path, SEQUENCE_LENGTH)
 
# Display the output video.
VideoFileClip(output_video_file_path, audio=False, target_resolution=(300,None)).ipython_display()

test_videos_1/test_video-Output-SeqLen40.mp4
1/1 [==============================] - 0s 12ms/step
[0.97496086 0.00570658 0.01933258] normal
1/1 [==============================] - 0s 12ms/step
[0.97232395 0.00598435 0.02169178] normal
1/1 [==============================] - 0s 11ms/step
[0.96937335 0.00629042 0.02433619] normal
1/1 [==============================] - 0s 11ms/step
[0.96698606 0.00651257 0.02650131] normal
1/1 [==============================] - 0s 11ms/step
[0.9653118  0.00667817 0.02800997] normal
1/1 [==============================] - 0s 11ms/step
[0.96438885 0.00676362 0.02884755] normal


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


1/1 [==============================] - 0s 11ms/step
[0.9640211  0.00679273 0.02918615] normal
1/1 [==============================] - 0s 11ms/step
[0.9638466  0.00679846 0.02935491] normal
1/1 [==============================] - 0s 11ms/step
[0.9639513  0.00677928 0.02926942] normal
1/1 [==============================] - 0s 11ms/step
[0.9641903  0.00675034 0.02905936] normal
1/1 [==============================] - 0s 12ms/step
[0.9645291  0.00671202 0.02875888] normal
1/1 [==============================] - 0s 12ms/step
[0.9648688  0.00666859 0.02846263] normal
1/1 [==============================] - 0s 11ms/step
[0.9652042  0.00662928 0.02816659] normal
1/1 [==============================] - 0s 12ms/step
[0.9654926  0.00659053 0.02791696] normal
1/1 [==============================] - 0s 11ms/step
[0.96579397 0.00655692 0.0276492 ] normal
1/1 [==============================] - 0s 11ms/step
[0.9660878  0.00651842 0.02739373] normal
1/1 [==============================] - 0s 12ms/step
[0.96636

1/1 [==============================] - 0s 12ms/step
[0.99680126 0.00104837 0.00215026] normal
1/1 [==============================] - 0s 12ms/step
[0.99677366 0.00104816 0.00217815] normal
1/1 [==============================] - 0s 11ms/step
[0.9967307  0.00104737 0.00222196] normal
1/1 [==============================] - 0s 11ms/step
[0.99674654 0.00103396 0.0022195 ] normal
1/1 [==============================] - 0s 11ms/step
[0.99678504 0.00101595 0.00219901] normal
1/1 [==============================] - 0s 11ms/step
[0.99679995 0.00100838 0.00219167] normal
1/1 [==============================] - 0s 11ms/step
[0.9968286  0.00100135 0.00216994] normal
1/1 [==============================] - 0s 12ms/step
[0.99681014 0.00100612 0.00218376] normal
1/1 [==============================] - 0s 11ms/step
[9.968713e-01 9.893555e-04 2.139372e-03] normal
1/1 [==============================] - 0s 11ms/step
[0.9967669  0.00102353 0.00220953] normal
1/1 [==============================] - 0s 11ms/step
[0

1/1 [==============================] - 0s 11ms/step
[0.9929905  0.00181272 0.00519674] normal
1/1 [==============================] - 0s 11ms/step
[0.9931485  0.00176229 0.00508917] normal
1/1 [==============================] - 0s 11ms/step
[0.99306834 0.00178801 0.00514357] normal
1/1 [==============================] - 0s 11ms/step
[0.99315095 0.00178718 0.00506192] normal
1/1 [==============================] - 0s 11ms/step
[0.9930599  0.00182339 0.00511671] normal
1/1 [==============================] - 0s 11ms/step
[0.9930108  0.00183633 0.00515279] normal
1/1 [==============================] - 0s 11ms/step
[0.9934041  0.00172538 0.0048705 ] normal
1/1 [==============================] - 0s 11ms/step
[0.99352694 0.00170396 0.00476919] normal
1/1 [==============================] - 0s 12ms/step
[0.9937781  0.0016341  0.00458773] normal
1/1 [==============================] - 0s 12ms/step
[0.99348384 0.00173423 0.0047819 ] normal
1/1 [==============================] - 0s 11ms/step
[0.99231

...

1/1 [==============================] - 0s 11ms/step
[0.990054   0.00341713 0.00652883] normal
1/1 [==============================] - 0s 12ms/step
[0.9912935  0.00299286 0.00571368] normal
1/1 [==============================] - 0s 11ms/step
[0.99253345 0.0025308  0.00493571] normal
1/1 [==============================] - 0s 11ms/step
[0.9924034  0.00261773 0.00497898] normal
1/1 [==============================] - 0s 11ms/step
[0.99271977 0.00265693 0.00462333] normal
1/1 [==============================] - 0s 11ms/step
[0.99209607 0.00278956 0.00511429] normal
1/1 [==============================] - 0s 11ms/step
[0.9918467  0.00288946 0.00526381] normal
1/1 [==============================] - 0s 11ms/step
[0.9920461  0.00276987 0.00518396] normal
1/1 [==============================] - 0s 11ms/step
[0.9921577  0.00248157 0.00536075] normal
1/1 [==============================] - 0s 11ms/step
[0.99173635 0.00223062 0.00603293] normal
1/1 [==============================] - 0s 12ms/step
[0.99123

1/1 [==============================] - 0s 12ms/step
[0.98899454 0.0015889  0.00941661] normal
1/1 [==============================] - 0s 11ms/step
[0.9889424  0.00185359 0.00920402] normal
1/1 [==============================] - 0s 11ms/step
[0.9892875  0.00201505 0.00869752] normal
1/1 [==============================] - 0s 11ms/step
[0.9892932  0.00217818 0.00852853] normal
1/1 [==============================] - 0s 11ms/step
[0.9898663  0.0022379  0.00789584] normal
1/1 [==============================] - 0s 11ms/step
[0.9907465  0.0021755  0.00707803] normal
1/1 [==============================] - 0s 11ms/step
[0.9917468  0.00219903 0.00605422] normal
1/1 [==============================] - 0s 11ms/step
[0.9915268  0.00244393 0.00602936] normal
1/1 [==============================] - 0s 11ms/step
[0.9911283  0.00268861 0.00618305] normal
1/1 [==============================] - 0s 11ms/step
[0.99119484 0.00283774 0.00596738] normal
1/1 [==============================] - 0s 11ms/step
[0.99053

1/1 [==============================] - 0s 14ms/step
[0.97194165 0.00425971 0.0237986 ] normal
1/1 [==============================] - 0s 14ms/step
[0.9671949  0.00451736 0.02828767] normal
1/1 [==============================] - 0s 14ms/step
[0.9607083  0.00486287 0.03442888] normal
1/1 [==============================] - 0s 14ms/step
[0.9518047 0.0053463 0.042849 ] normal
1/1 [==============================] - 0s 14ms/step
[0.9398507  0.00593011 0.05421924] normal
1/1 [==============================] - 0s 14ms/step
[0.9235236  0.00666244 0.06981389] normal
1/1 [==============================] - 0s 14ms/step
[0.9010861  0.00756036 0.09135353] normal
1/1 [==============================] - 0s 13ms/step
[0.86932164 0.00866543 0.12201294] normal
1/1 [==============================] - 0s 13ms/step
[0.8330523  0.00968815 0.15725963] normal
1/1 [==============================] - 0s 14ms/step
[0.8075097  0.01031478 0.18217556] normal
1/1 [==============================] - 0s 13ms/step
[0.7722755 

Moviepy - Done !
Moviepy - video ready __temp__.mp4


ValueError: The duration of video __temp__.mp4 (81.6) exceeds the 'maxduration' attribute. You can increase 'maxduration', by passing 'maxduration' parameterto ipython_display function.But note that embedding large videos may take all the memory away !